In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import transect_viz
from transect_viz import transect_cdec_data
import pandas as pd

In [ ]:
df = transect_viz.load_transect_file('../data/Transect_20220302_Fabian.csv')
df['EC'] = df['Sonde1_SpCond']
df.head(3)

In [ ]:
import shapely

In [ ]:
from shapely.geometry import Point, LineString

In [ ]:
lonlats=df[['Longitude','Latitude']].values

In [ ]:
transect_line = LineString(coordinates=lonlats)

In [ ]:
dfs = transect_cdec_data.get_stations_cached(['ORM','ODM','OLD','GLE','GLC'])

In [ ]:
dfs

In [ ]:
transect_line.simplify(tolerance=0.001)

In [ ]:
p3 = Point(dfs.loc[3,['Longitude','Latitude']].values)

In [ ]:
print(p3)

In [ ]:
transect_line.project(p3)

In [ ]:
print(transect_line.interpolate(transect_line.project(p3)))

In [ ]:
from shapely.geometry import LinearRing

In [ ]:
transect_ring = LinearRing(transect_line.coords)

In [ ]:
from shapely.ops import nearest_points

In [ ]:
for x in nearest_points(transect_ring, p3): print(x)

In [ ]:
import geopandas as gpd

In [ ]:
gdf = gpd.GeoDataFrame(df)

In [ ]:
gdf['geometry']=gpd.points_from_xy(df['Longitude'],df['Latitude'],crs="EPSG:4326")

In [ ]:
gdf = gdf.to_crs('EPSG:32610')

In [ ]:
utmxy = gdf.geometry

In [ ]:
transect_line = LineString(utmxy)

In [ ]:
transect_line.project(utmxy.iloc[-1])

In [ ]:
cdec_stations=['GLE','GLC','ORM','ODM','OLD']

In [ ]:
dfs = transect_cdec_data.get_stations_cached(cdec_stations)

In [ ]:
dfs

In [ ]:
transect_line.project(Point(dfs.loc[0,['Longitude','Latitude']]))

In [ ]:
gdfs = gpd.GeoDataFrame(dfs, geometry=gpd.points_from_xy(dfs['Longitude'],dfs['Latitude'],crs='EPSG:4326'))

In [ ]:
gdfs

In [ ]:
transect_line.project(gdfs.geometry[0])

In [ ]:
gdfs = gdfs.to_crs('EPSG:32610')

In [ ]:
gdfs

In [ ]:
transect_line.project(gdfs.geometry[2])

In [ ]:
gdfs['transect_dist']=[transect_line.project(row.geometry) for _,row in gdfs.iterrows()]
    

In [ ]:
gdfs.sort_values(by='transect_dist')

In [ ]:
bsdate, bedate = transect_viz.get_buffered_start_end_dates(df)

In [ ]:
import pandas as pd

In [ ]:
df_cdec_15 = transect_cdec_data.get_cdec_data_cached(bsdate, bedate, cdec_stations,data_type='EC')

In [ ]:
df.DateTime

In [ ]:
df_cdec_15_1min = df_cdec_15.resample('T').interpolate()

In [ ]:
df_cdec_15_1min.loc[df.DateTime.iloc[0]]

In [ ]:
df_cdec_15_1min.loc[df.DateTime.unique()]

In [ ]:
gdf['transect_dist'] = [transect_line.project(row.geometry) for _,row in gdf.iterrows()]

In [ ]:
gdf

In [ ]:
import numpy as np

In [ ]:
gdf = gdf.astype({'DateTime':np.datetime64})

In [ ]:
gdf.join(df_cdec_15_1min)

In [ ]:
df_cdec_15_1min

In [ ]:
gdf.set_index('DateTime').join(df_cdec_15_1min)

In [ ]:
transect_max, transect_min = gdf.transect_dist.agg(['max','min'])

In [ ]:
df_cdec_15

In [ ]:
gdfs = gdfs.sort_values(by='transect_dist').reset_index(drop=True)
gdfs

In [ ]:
pd.concat([pd.DataFrame(data=[['BEG',37.803,-121.475,np.nan,0]], columns=gdfs.columns),gdfs])

In [ ]:
pgdf = gdf[['Latitude','Longitude','Sonde1_SpCond','transect_dist']]

In [ ]:
df_end_points = pd.DataFrame(pgdf.iloc[[0,-1]])
df_end_points['Station ID']=['BEG','END']
df_end_points

In [ ]:
df_ec_linear_refs=pd.concat([df_end_points,gdfs]).sort_values('transect_dist').reset_index(drop=True)
df_ec_linear_refs

In [ ]:
df_ec_linear_refs.iloc[1:6]

In [ ]:
df_ec_vals = df_cdec_15.iloc[0:1].T
df_ec_vals.columns=['Obs EC']
df_ec_vals

In [ ]:
df_ec_at_stations=df_ec_vals.join(df_ec_linear_refs.set_index('Station ID')).sort_values(by='transect_dist')

In [ ]:
df_ec_at_stations

In [ ]:
df_ec_at_stations.loc['OLD']['transect_dist'], transect_max

In [ ]:
factor = df_ec_at_stations.loc['OLD']['transect_dist']/(transect_max-df_ec_at_stations.loc['ORM']['transect_dist'])

In [ ]:
ec_beg_end = df_ec_at_stations.loc['OLD']['Obs EC'] + factor * (df_ec_at_stations.loc['OLD']['Obs EC']-df_ec_at_stations.loc['ORM']['Obs EC'])

In [ ]:
ec_beg_end

In [ ]:
df_ec_linear_refs

In [ ]:
df_ec_linear_refs.loc[df_ec_linear_refs['Station ID']=='BEG','Sonde1_SpCond'] = ec_beg_end

In [ ]:
df_ec_linear_refs.loc[df_ec_linear_refs['Station ID']=='END','Sonde1_SpCond'] = ec_beg_end

In [ ]:
df_ec_linear_refs = df_ec_linear_refs.set_index('Station ID').join(df_ec_vals).reset_index()
df_ec_linear_refs

In [ ]:
df_ec_linear_refs['Sonde1_SpCond']=df_ec_linear_refs['Sonde1_SpCond'].fillna(df_ec_linear_refs['Obs EC'])

In [ ]:
df_ec_linear_refs

In [ ]:
columns = ['Longitude','Latitude','Sonde1_SpCond','transect_dist']

In [ ]:
dfcalc = df_ec_linear_refs[columns].copy()

In [ ]:
dfcalc.loc[:,'Sonde1_SpCond']=np.nan

In [ ]:
dfcalc

In [ ]:
dfmerged=pd.concat([dfcalc,gdf[columns]]).sort_values(by='transect_dist').reset_index(drop=True)

In [ ]:
dfmerged

In [ ]:
dfresult = dfmerged.interpolate().dropna()

In [ ]:
import hvplot.pandas

In [ ]:
dfresult['DateTime']='2022-02-28 00:00'

In [ ]:
dfresult

In [ ]:
map, legend = transect_viz.map_transect_with_size_and_color(dfresult, data_column='Sonde1_SpCond')

In [ ]:
carto_light_tiles=transect_viz.get_tile_layer()

In [ ]:
carto_light_tiles*map.opts(frame_width=800, colorbar=True)+legend